### hw print 결과들을 pandas Dataframe으로 전처리하는 코드 입니다.

In [1]:
import glob
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
# file_path는 실험결과가 어디에 저장되는지에 따라 달라질 수 있습니다.
dir_path = "/home/centos/firesim/deploy/results-workload/allocate-fpga-print/version3/"
file_paths = glob.glob(dir_path+'/*/syn*')
file_paths = sorted(file_paths)
file_paths

['/home/centos/firesim/deploy/results-workload/allocate-fpga-print/version3/soc-gemmini-multisim.2MB-gemmini.basic/synthesized-prints.out0',
 '/home/centos/firesim/deploy/results-workload/allocate-fpga-print/version3/soc-gemmini-multisim.2MB-gemmini.double/synthesized-prints.out0',
 '/home/centos/firesim/deploy/results-workload/allocate-fpga-print/version3/soc-gemmini-multisim.2MB-gemmini.loop/synthesized-prints.out0',
 '/home/centos/firesim/deploy/results-workload/allocate-fpga-print/version3/soc-gemmini-multisim.gemmini.basic/synthesized-prints.out0',
 '/home/centos/firesim/deploy/results-workload/allocate-fpga-print/version3/soc-gemmini-multisim.gemmini.double/synthesized-prints.out0',
 '/home/centos/firesim/deploy/results-workload/allocate-fpga-print/version3/soc-gemmini-multisim.gemmini.loop/synthesized-prints.out0']

In [3]:
if not os.path.isdir("./dataframe"):
	os.mkdir("./dataframe")

In [6]:
for file_path in file_paths:
	line_count = 0
	
	# line count
	with open(file_path, 'r') as f:
		one_line = f.readline()
		while True:
			one_line = f.readline()
			if not one_line:
				break
			line_count+=1

	# pre-allocate memory space
	cycle = np.zeros(line_count, dtype=np.uint32)
	control = np.zeros(line_count, dtype=np.str)
	bank = np.zeros(line_count, dtype=np.uint8)
	row_addr = np.zeros(line_count, dtype=np.uint16)

	# read text file
	with open(file_path, 'r') as f:
		one_line = f.readline() #skip first line
		for i in range(line_count):
			one_line = f.readline()
			if not one_line:
				break

			str1, *str2 = one_line.split(' ')[3:]
			cycle[i] = int(str1)
			val1, val2, val3 = ''.join(str2).strip().split(',')
			control[i] = val1
			bank[i] = int(val2)
			row_addr[i] = int(val3)
	
	# make DataFrame and check conflict
	df = pd.DataFrame({"cycle":cycle,"control":control,"bank":bank, "row_addr":row_addr})
	df_e = df[df.control=='E']
	df_l = df[df.control=='L']
	df_all = pd.merge(left=df_l, right=df_e, how="outer", on='cycle')

	bool_idx = df_all.isna()
	bool_idx =  np.all(~bool_idx, axis = 1)

	df_both = df_all[bool_idx]
	df_both = df_both.astype({"bank_x":np.uint8,"bank_y":np.uint8})
	df_conflict = df_both[df_both.bank_x==df_both.bank_y]

	#print total_cycle, conflict_cycle
	print(df.iloc[-1].cycle - df.iloc[0].cycle, len(df_conflict))

	bool_idx = np.logical_and(df.cycle.isin(df_conflict.cycle), df.control=="L")
	df.loc[bool_idx, 'control'] ='LC'
	bool_idx = np.logical_and(df.cycle.isin(df_conflict.cycle), df.control=="E")
	df.loc[bool_idx, 'control'] ='EC'

	print(f"{file_path} completed")


	# save pickle
	with open("dataframe/"+file_path.split('/')[-2], 'wb') as f:
		pickle.dump(df, f)

KeyboardInterrupt: 

In [ ]:
# result                         (total_cycle, conflict_cycle)

# 2MB-gemmini.basic           --> (474464744, 2960975)
# 2MB-gemmini.double          --> (473868989, 2946508)
# 2MB-gemmini.loop            --> (131033537, 3415504)

# gemmini.basic               --> (467702055, 2622277)
# gemmini.double              --> (467229573, 2632215)
# gemmini.loop                --> (123235541, 3710762)